In [1]:
import pandas as pd
import numpy as np

In [ ]:
# Load datasets
df_train = pd.read_csv("data/train.csv")
df_test = pd.read_csv("data/test_mini.csv")

In [3]:
df_train.drop(['id'], axis=1, inplace=True)
df_test.drop(['id'], axis=1, inplace=True)

In [4]:
def drop_duplicate_cols(df):
    dup_mask = df.T.duplicated()
    return df.loc[:, ~dup_mask]

df_train_unique = drop_duplicate_cols(df_train)
df_test_unique = drop_duplicate_cols(df_test)

print("df_train:", df_train.shape, "->", df_train_unique.shape)
print("df_test: ", df_test.shape,  "->", df_test_unique.shape)

df_train: (76905, 7594) -> (76905, 3109)
df_test:  (1001, 7591) -> (1001, 255)


In [5]:
# Compute correlation matrices on numeric columns
#df_train_corr = df_train_unique.select_dtypes(include="number").corr()
#df_test_corr  = df_test_unique.select_dtypes(include="number").corr()

# Display the correlation matrices
#df_train_corr
#df_test_corr

In [6]:
df_train = df_train_unique.loc[:, df_train.nunique() > 1]
df_test = df_test_unique.loc[:, df_test.nunique() > 1]

In [7]:
df_corr = df_test.corr()

In [8]:
df_corr

,demand_2_0,demand_2_1,demand_9_0,demand_9_2,demand_9_4,demand_9_5,demand_10_6,demand_10_7,demand_11_1,demand_11_2,...,param_year0_f13,param_year0_f14,param_year0_f15,param_year0_f16,param_year1_f14,param_year1_f15,param_year1_f16,param_year2_f14,param_year2_f15,param_year2_f16
demand_2_0,1.000000,-1.000000,0.901400,-0.901400,-0.049844,0.049844,-0.397410,0.397410,-0.900364,0.900364,...,0.004005,0.053012,0.068046,-0.254088,0.078080,0.066081,-0.079681,0.075499,0.062451,-0.039565
demand_2_1,-1.000000,1.000000,-0.901400,0.901400,0.049844,-0.049844,0.397410,-0.397410,0.900364,-0.900364,...,-0.004005,-0.053012,-0.068046,0.254088,-0.078080,-0.066081,0.079681,-0.075499,-0.062451,0.039565
demand_9_0,0.901400,-0.901400,1.000000,-1.000000,0.387520,-0.387520,-0.358226,0.358226,-0.811588,0.811588,...,-0.003768,0.087601,0.056458,-0.182878,0.101860,0.042401,-0.070344,0.099227,0.039546,-0.040488
demand_9_2,-0.901400,0.901400,-1.000000,1.000000,-0.387520,0.387520,0.358226,-0.358226,0.811588,-0.811588,...,0.003768,-0.087601,-0.056458,0.182878,-0.101860,-0.042401,0.070344,-0.099227,-0.039546,0.040488
demand_9_4,-0.049844,0.049844,0.387520,-0.387520,1.000000,-1.000000,0.019809,-0.019809,0.044878,-0.044878,...,-0.017218,0.089200,-0.014647,0.119133,0.068718,-0.042886,0.007389,0.068142,-0.041744,-0.009156
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
param_year1_f15,0.066081,-0.066081,0.042401,-0.042401,-0.042886,0.042886,0.016758,-0.016758,-0.079896,0.079896,...,-0.019025,-0.018485,0.758875,-0.060438,-0.008490,1.000000,0.112253,-0.016542,0.744639,0.188539
param_year1_f16,-0.079681,0.079681,-0.070344,0.070344,0.007389,-0.007389,0.078553,-0.078553,0.049509,-0.049509,...,0.029897,0.036300,0.141526,0.303419,0.055522,0.112253,1.000000,0.021310,0.155841,0.652436
param_year2_f14,0.075499,-0.075499,0.099227,-0.099227,0.068142,-0.068142,-0.135821,0.135821,-0.017799,0.017799,...,0.101063,0.754659,0.012967,-0.239887,0.722034,-0.016542,0.021310,1.000000,-0.021745,0.020385
param_year2_f15,0.062451,-0.062451,0.039546,-0.039546,-0.041744,0.041744,0.013488,-0.013488,-0.074394,0.074394,...,-0.027132,-0.019585,0.742565,-0.045354,-0.010989,0.744639,0.155841,-0.021745,1.000000,0.220423


In [9]:

df_corr = df_corr.abs()

upper = df_corr.where(np.triu(np.ones(df_corr.shape), k=1).astype(bool))

threshold = 0.90
to_drop = [column for column in upper.columns if any(upper[column] > threshold)]

cols_to_drop_train = [c for c in to_drop if c in df_train.columns]
cols_to_drop_test = [c for c in to_drop if c in df_test.columns]

df_train_reduced = df_train.drop(columns=cols_to_drop_train)
df_test_reduced = df_test.drop(columns=cols_to_drop_test)

print("Reduced shape:", df_train_reduced.shape)
print(f"Requested to drop: {len(to_drop)} columns")
print(f"Actually dropped in train: {len(cols_to_drop_train)}")
print("Dropped columns (train):", cols_to_drop_train)

Reduced shape: (76905, 2971)
Requested to drop: 136 columns
Actually dropped in train: 136
Dropped columns (train): ['demand_2_1', 'demand_9_0', 'demand_9_2', 'demand_9_5', 'demand_10_7', 'demand_11_1', 'demand_11_2', 'demand_44_1', 'demand_54_0', 'demand_54_1', 'demand_66_2', 'demand_66_5', 'demand_69_1', 'demand_103_1', 'demand_106_4', 'demand_132_4', 'demand_188_1', 'demand_217_1', 'demand_219_5', 'demand_220_0', 'demand_221_5', 'demand_222_1', 'demand_222_5', 'demand_222_6', 'demand_223_4', 'demand_223_9', 'demand_224_0', 'demand_224_2', 'demand_224_6', 'demand_224_7', 'demand_225_0', 'demand_225_2', 'demand_227_1', 'demand_227_6', 'demand_228_6', 'demand_229_6', 'demand_229_8', 'demand_235_5', 'demand_237_4', 'demand_237_5', 'demand_237_9', 'demand_239_4', 'demand_239_6', 'demand_240_1', 'demand_240_5', 'demand_244_6', 'demand_246_6', 'demand_249_0', 'demand_249_1', 'demand_254_2', 'demand_254_7', 'demand_255_7', 'demand_258_2', 'demand_263_1', 'demand_263_3', 'demand_263_6', 'dem

In [10]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score, mean_squared_error

targets = ['wip', 'investissement', 'satisfaction']

X = df_train_reduced.drop(columns=targets)
y = df_train_reduced[targets]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
'''
model = LinearRegression()
model.fit(X_train, y_train)

y_pred = model.predict(X_test)

r2_per_target = r2_score(y_test, y_pred, multioutput='raw_values')
r2_overall = r2_score(y_test, y_pred, multioutput='uniform_average')
rmse_per_target = np.sqrt(mean_squared_error(y_test, y_pred, multioutput='raw_values'))
mae_per_target = np.mean(np.abs(y_test - y_pred), axis=0)

for t, r, rm, mae in zip(targets, r2_per_target, rmse_per_target, mae_per_target):
    print(f"{t}: R2={r:.4f}, RMSE={rm:.4f}, MAE={mae:.4f}")
print(f"Overall R2: {r2_overall:.4f}")
'''

'\nmodel = LinearRegression()\nmodel.fit(X_train, y_train)\n\ny_pred = model.predict(X_test)\n\nr2_per_target = r2_score(y_test, y_pred, multioutput=\'raw_values\')\nr2_overall = r2_score(y_test, y_pred, multioutput=\'uniform_average\')\nrmse_per_target = np.sqrt(mean_squared_error(y_test, y_pred, multioutput=\'raw_values\'))\nmae_per_target = np.mean(np.abs(y_test - y_pred), axis=0)\n\nfor t, r, rm, mae in zip(targets, r2_per_target, rmse_per_target, mae_per_target):\n    print(f"{t}: R2={r:.4f}, RMSE={rm:.4f}, MAE={mae:.4f}")\nprint(f"Overall R2: {r2_overall:.4f}")\n'

In [11]:
from sklearn.ensemble import RandomForestRegressor
'''
rf = RandomForestRegressor(n_estimators=100, random_state=42, n_jobs=-1)
rf.fit(X_train, y_train)

y_pred = rf.predict(X_test)

r2_per_target = r2_score(y_test, y_pred, multioutput='raw_values')
r2_overall = r2_score(y_test, y_pred, multioutput='uniform_average')
rmse_per_target = np.sqrt(mean_squared_error(y_test, y_pred, multioutput='raw_values'))
mae_per_target = np.mean(np.abs(y_test - y_pred), axis=0)

for t, r, rm, mae in zip(targets, r2_per_target, rmse_per_target, mae_per_target):
    print(f"{t}: R2={r:.4f}, RMSE={rm:.4f}, MAE={mae:.4f}")
print(f"Overall R2: {r2_overall:.4f}")
'''

'\nrf = RandomForestRegressor(n_estimators=100, random_state=42, n_jobs=-1)\nrf.fit(X_train, y_train)\n\ny_pred = rf.predict(X_test)\n\nr2_per_target = r2_score(y_test, y_pred, multioutput=\'raw_values\')\nr2_overall = r2_score(y_test, y_pred, multioutput=\'uniform_average\')\nrmse_per_target = np.sqrt(mean_squared_error(y_test, y_pred, multioutput=\'raw_values\'))\nmae_per_target = np.mean(np.abs(y_test - y_pred), axis=0)\n\nfor t, r, rm, mae in zip(targets, r2_per_target, rmse_per_target, mae_per_target):\n    print(f"{t}: R2={r:.4f}, RMSE={rm:.4f}, MAE={mae:.4f}")\nprint(f"Overall R2: {r2_overall:.4f}")\n'

In [12]:
from sklearn.ensemble import GradientBoostingRegressor, ExtraTreesRegressor
from sklearn.svm import SVR
from sklearn.neighbors import KNeighborsRegressor
from sklearn.multioutput import MultiOutputRegressor
from sklearn.linear_model import Ridge
from sklearn.metrics import r2_score, mean_squared_error
import numpy as np
import xgboost as xgb
import pandas as pd

candidates = [
    #("Linear", LinearRegression()),
    #("Ridge", MultiOutputRegressor(Ridge())),
    #("RandomForest", RandomForestRegressor(n_estimators=200, random_state=42, n_jobs=-1)),
    #("ExtraTrees", ExtraTreesRegressor(n_estimators=200, random_state=42, n_jobs=-1)),
    #("GradientBoosting", MultiOutputRegressor(GradientBoostingRegressor(random_state=42))),
    #("KNeighbors", MultiOutputRegressor(KNeighborsRegressor())),
    #("SVR", MultiOutputRegressor(SVR()))
]

try:
    candidates.append(("XGB", MultiOutputRegressor(xgb.XGBRegressor(objective='reg:squarederror', n_estimators=100, random_state=42, n_jobs=-1))))
except Exception:
    pass

results = []
predictions = {}

for name, model in candidates:
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    predictions[name] = y_pred
    r2_per = r2_score(y_test, y_pred, multioutput='raw_values')
    r2_overall = r2_score(y_test, y_pred, multioutput='uniform_average')
    rmse_per = np.sqrt(mean_squared_error(y_test, y_pred, multioutput='raw_values'))
    mae_per = np.mean(np.abs(y_test - y_pred), axis=0)
    results.append({
        "model": name,
        "r2_overall": r2_overall,
        "r2_per": r2_per,
        "rmse_per": rmse_per,
        "mae_per": mae_per
    })
    print(f"{name}: Overall R2={r2_overall:.4f}")

summary = pd.DataFrame([{"model": r["model"], "r2_overall": r["r2_overall"]} for r in results]).sort_values("r2_overall", ascending=False)
print("\nModel ranking (by overall R2):")
print(summary.reset_index(drop=True))

# Simple averaging ensemble of top 3 models
top3 = summary["model"].values[:3].tolist()
avg_pred = np.mean([predictions[m] for m in top3], axis=0)
r2_avg = r2_score(y_test, avg_pred, multioutput='uniform_average')
print(f"\nAveraging ensemble (top3={top3}): Overall R2={r2_avg:.4f}")


r2_matrix = np.vstack([r["r2_per"] for r in results])  # shape (n_models, n_targets)
best_per_target_idx = np.argmax(r2_matrix, axis=0)
per_target_pred = np.zeros_like(avg_pred)
for ti, midx in enumerate(best_per_target_idx):
    per_target_pred[:, ti] = predictions[results[midx]["model"]][:, ti]
r2_per_target_ensemble = r2_score(y_test, per_target_pred, multioutput='raw_values')
r2_per_target_overall = r2_score(y_test, per_target_pred, multioutput='uniform_average')
print(f"Per-target best-model ensemble: Overall R2={r2_per_target_overall:.4f}")
for t, r in zip(targets, r2_per_target_ensemble):
    print(f"  {t}: R2={r:.4f}")

XGB: Overall R2=0.8835

Model ranking (by overall R2):
  model  r2_overall
0   XGB    0.883495

Averaging ensemble (top3=['XGB']): Overall R2=0.8835
Per-target best-model ensemble: Overall R2=0.8835
  wip: R2=0.7281
  investissement: R2=1.0000
  satisfaction: R2=0.9224


In [13]:

for r in results:
    name = r["model"]
    r2_overall = r["r2_overall"]
    rmse_per = r["rmse_per"]
    mae_per = r["mae_per"]
    print(f"{name} Errors:")
    for t, rmse, mae in zip(targets, rmse_per, mae_per):
        print(f"  {t}: RMSE={rmse:.4f}, MAE={mae:.4f}")
    print(f"  Overall R2={r2_overall:.4f}\n")

XGB Errors:
  wip: RMSE=2416809.7500, MAE=1612660.8831
  investissement: RMSE=0.0890, MAE=0.0840
  satisfaction: RMSE=0.0577, MAE=0.0425
  Overall R2=0.8835



In [14]:
df_test_final = pd.read_csv("test.csv")

In [15]:
# prepare test features to match training features and predict with the fitted model
ids = df_test_final['id'].copy()
X_test_final = df_test_final.drop(columns=['id']).reindex(columns=X.columns, fill_value=0)
X_test_final = X_test_final[X.columns]  # ensure same column order

preds = model.predict(X_test_final)  # shape (n_samples, 3)

df_submission = pd.DataFrame(preds, columns=['wip', 'investissement', 'satisfaction'])
df_submission.insert(0, 'id', ids.values)

print(df_submission.head())
# df_submission is the result with columns ['id','wip','investissement','satisfaction']

# exporting it to csv
df_submission.to_csv("submission.csv", index=False)

   id         wip  investissement  satisfaction
0   0  27126452.0    1.000000e+06      0.389340
1   1  22676116.0    1.250000e+06      0.498064
2   2  24983442.0    7.500001e+05      0.601970
3   3  26029758.0    1.250000e+06      0.812852
4   4  28487360.0    1.125000e+06      0.801151


In [23]:
y_test

,wip,investissement,satisfaction
76828,2.916826e+07,875000.0,0.953826
73881,2.997447e+07,1250000.0,0.761158
2009,3.564799e+07,875000.0,0.846764
26610,2.022953e+07,1125000.0,0.534519
70500,2.836328e+07,1000000.0,0.950798
...,...,...,...
53278,2.773897e+07,1000000.0,0.969415
151,1.508658e+07,1125000.0,0.542074
29154,2.145134e+07,1000000.0,0.351184
64557,2.387047e+07,750000.0,0.549161


In [28]:
# prédictions -> DataFrame
df_pred = pd.DataFrame(y_pred[:, 2], columns=["satisfaction"], index=y_test['satisfaction'].index)
# vraies valeurs
df_true = pd.DataFrame(y_test['satisfaction'].copy())

# errreur
df_error = (df_true - df_pred).abs()

# tableau final
df_results = pd.concat([df_true, df_pred.add_suffix("_pred"), df_error.add_suffix("_error")], axis=1)

df_results.head()

,satisfaction,satisfaction_pred,satisfaction_error
76828,0.953826,0.944005,0.009821
73881,0.761158,0.705938,0.055221
2009,0.846764,0.827003,0.019760
26610,0.534519,0.666645,0.132126
70500,0.950798,0.924248,0.026550


In [29]:
mask = df_results["satisfaction_error"] < 0.05

count_ok = mask.sum()
ratio_ok = mask.mean()  

count_ok, ratio_ok

(np.int64(10639), np.float64(0.6916975489239972))